In [2]:
import pandas as pd

In [3]:
anime_ratings_df = pd.read_csv("datasets/anime/rating.csv")
anime_ratings_df.shape
print(anime_ratings_df.head())

   user_id  anime_id  rating
0        1        20      -1
1        1        24      -1
2        1        79      -1
3        1       226      -1
4        1       241      -1


In [4]:
anime_ratings = anime_ratings_df.loc[anime_ratings_df.rating != -1].reset_index()[['user_id','anime_id','rating']]

In [5]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import random
import implicit

In [6]:
import random
from sklearn.preprocessing import MinMaxScaler

In [7]:
def encode_column(column):
    """ Encodes a pandas column with continous IDs"""
    keys = column.unique()
    key_to_id = {key:idx for idx,key in enumerate(keys)}
    return key_to_id, np.array([key_to_id[x] for x in column]), len(keys)

In [8]:
def encode_df(anime_df):
    """Encodes rating data with continuous user and anime ids"""
    
    anime_ids, anime_df['anime_id'], num_anime = encode_column(anime_df['anime_id'])
    user_ids, anime_df['user_id'], num_users = encode_column(anime_df['user_id'])
    return anime_df, num_users, num_anime, user_ids, anime_ids

In [9]:
anime_df, num_users, num_anime, user_ids, anime_ids = encode_df(anime_ratings)


In [10]:
anime_df

,user_id,anime_id,rating
0,0,0,10
1,0,1,10
2,0,2,10
3,0,3,10
4,1,4,10
...,...,...,...
6337236,69598,56,7
6337237,69598,1802,9
6337238,69598,783,10
6337239,69599,753,9


In [11]:
sparse_item_user = sparse.csr_matrix((anime_df['rating'].values,(anime_df['user_id'].values, anime_df['anime_id'].values)),shape=(num_users, num_anime))

In [11]:
sparse_user_item = sparse.csr_matrix((anime_df['rating'].values,(anime_df['anime_id'].values, anime_df['user_id'].values)),shape=(num_anime, num_users))

In [12]:
model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.01, iterations=20)

In [12]:
from sklearn.model_selection import KFold, train_test_split

In [14]:
alpha_val = 40
data_conf = (sparse_item_user * alpha_val).astype('double')

In [17]:
train_matrix, test_matrix = train_test_split(data_conf)

In [22]:
model.fit(train_matrix)

  0%|          | 0/20 [00:00<?, ?it/s]

In [13]:
from implicit.evaluation import ndcg_at_k

In [14]:
from implicit.evaluation import precision_at_k

In [15]:
from implicit.evaluation import mean_average_precision_at_k

In [28]:
eval_als(model, train_matrix, test_matrix)

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

...

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

-------- K = 1 --------
NDCG@k:	0.097816
Recall@k:	0.097816
MNAP@K:	0.097816
-------- K = 5 --------
NDCG@k:	0.100579
Recall@k:	0.107920
MNAP@K:	0.069687
-------- K = 10 --------
NDCG@k:	0.103228
Recall@k:	0.116659
MNAP@K:	0.059904
-------- K = 15 --------
NDCG@k:	0.106354
Recall@k:	0.124860
MNAP@K:	0.055404
-------- K = 20 --------
NDCG@k:	0.109209
Recall@k:	0.131985
MNAP@K:	0.052965


In [23]:
titles = pd.read_csv("anime.csv")

In [53]:
titles.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [19]:
from implicit.evaluation import leave_k_out_split, precision_at_k, train_test_split

In [20]:
recalculated_ids, recalculated_scores = model.recommend(
        userid, sparse_item_user[userid], N=3, recalculate_user=True
    )

In [21]:
    top_rec, score = recalculated_ids[0], recalculated_scores[0]

In [22]:
score_explained, contributions, W = model.explain(userid, sparse_item_user, itemid=top_rec)
scores = [s for _, s in contributions]
items = [i for i, _ in contributions]

In [23]:
score, score_explained

(0.6470378, 0.647037919544727)

In [ ]:
scores

[0.015420449005689535,
 0.015182762272978257,
 0.01511981341493532,
 0.014851731425630387,
 0.014648284435983766,
 0.014034782127265737,
 0.013941028812920945,
 0.013812924224395125,
 0.013599989023189004,
 0.013481741508972621]

In [101]:
model.evaluate()

AttributeError: 'AlternatingLeastSquares' object has no attribute 'evaluate'

In [40]:
model2 = implicit.bpr.BayesianPersonalizedRanking(factors = 20, regularization=0.1, iterations=20)

In [41]:
model2.fit(train_matrix)

  0%|          | 0/20 [00:00<?, ?it/s]

In [16]:
def eval_bpr(model,train, test):
    from implicit.evaluation import ndcg_at_k as ndcg_imp
    from implicit.evaluation import mean_average_precision_at_k
    from implicit.evaluation import precision_at_k as pr_imp

    return print("-------- K = 1 --------",
                  "NDCG@k:\t%f" % ndcg_imp(model,train, test, K=1),
                  "Recall@k:\t%f" % pr_imp(model,train, test, K=1),
                  "MNAP@K:\t%f" % mean_average_precision_at_k(model,train, test, K=1), 
                  "-------- K = 5 --------",
                  "NDCG@k:\t%f" % ndcg_imp(model,train, test, K=5),
                  "Recall@k:\t%f" % pr_imp(model,train, test, K=5),
                  "MNAP@K:\t%f" % mean_average_precision_at_k(model,train, test, K=5), 
                  "-------- K = 10 --------",
                  "NDCG@k:\t%f" % ndcg_imp(model,train, test, K=10),
                  "Recall@k:\t%f" % pr_imp(model,train, test, K=10),
                  "MNAP@K:\t%f" % mean_average_precision_at_k(model,train, test, K=10), 
                  "-------- K = 15 --------",
                  "NDCG@k:\t%f" % ndcg_imp(model,train, test, K=15),
                  "Recall@k:\t%f" % pr_imp(model,train, test, K=15),
                  "MNAP@K:\t%f" % mean_average_precision_at_k(model,train, test, K=15), 
                  "-------- K = 20 --------",
                  "NDCG@k:\t%f" % ndcg_imp(model,train, test, K=20),
                  "Recall@k:\t%f" % pr_imp(model,train, test, K=20),
                  "MNAP@K:\t%f" % mean_average_precision_at_k(model,train, test, K=20), sep = '\n')

In [48]:
eval_bpr(model2,train_matrix, test_matrix)

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

...

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

  0%|          | 0/17400 [00:00<?, ?it/s]

-------- K = 1 --------
NDCG@k:	0.373276
Recall@k:	0.373276
MNAP@K:	0.373276
-------- K = 5 --------
NDCG@k:	0.307713
Recall@k:	0.311045
MNAP@K:	0.232278
-------- K = 10 --------
NDCG@k:	0.285390
Recall@k:	0.296084
MNAP@K:	0.185644
-------- K = 15 --------
NDCG@k:	0.272327
Recall@k:	0.287780
MNAP@K:	0.162787
-------- K = 20 --------
NDCG@k:	0.262579
Recall@k:	0.280874
MNAP@K:	0.147870


In [17]:
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions

from recommenders.datasets.python_splitters import python_random_split
import surprise

In [20]:
svd = surprise.SVD(random_state=0, n_factors=200, n_epochs=10, verbose=True)

In [21]:
anime_df = anime_df.rename(columns = {'user_id':'userID','anime_id':'itemID'})

In [22]:
anime_df.head()

,userID,itemID,rating
0,0,0,10
1,0,1,10
2,0,2,10
3,0,3,10
4,1,4,10


In [23]:
train, test = python_random_split(anime_df, 0.75)

In [24]:
from surprise import Reader

In [25]:
reader = Reader(rating_scale=(1, 10))

In [26]:
train_set = surprise.Dataset.load_from_df(train, reader).build_full_trainset()

In [27]:
svd = surprise.SVD(random_state=0, n_factors=200, n_epochs=10, verbose=True)

In [28]:
svd.fit(train_set)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [29]:
def eval_svd(model,train, test, usercol, itemcol):
    all_predictions = compute_ranking_predictions(model, train, usercol=usercol, itemcol=itemcol, remove_seen=True)
    return print("-------- K = 1 --------",
                  "NDCG@k:\t%f" % ndcg_at_k(test, all_predictions, col_user = usercol, col_item = itemcol, col_rating = 'Rating', col_prediction='prediction', k=1),
                  "Recall@k:\t%f" % precision_at_k(test, all_predictions, col_user = usercol, col_item = itemcol, col_rating = 'Rating', col_prediction='prediction', k=1),
                  "MNAP@K:\t%f" % mean_average_precision_at_k(model,train, test, K=1), 
                  "-------- K = 5 --------",
                  "NDCG@k:\t%f" % ndcg_at_k(test, all_predictions, col_user = usercol, col_item = itemcol, col_rating = 'Rating', col_prediction='prediction', k=5),
                  "Recall@k:\t%f" % precision_at_k(test, all_predictions, col_user = usercol, col_item = itemcol, col_rating = 'Rating', col_prediction='prediction', k=5),
                  "MNAP@K:\t%f" % mean_average_precision_at_k(model,train, test, K=5), 
                  "-------- K = 10 --------",
                  "NDCG@k:\t%f" % ndcg_at_k(test, all_predictions, col_user = usercol, col_item = itemcol, col_rating = 'Rating', col_prediction='prediction', k=10),
                  "Recall@k:\t%f" % precision_at_k(test, all_predictions, col_user = usercol, col_item = itemcol, col_rating = 'Rating', col_prediction='prediction', k=10),
                  "MNAP@K:\t%f" % mean_average_precision_at_k(model,train, test, K=10), 
                  "-------- K = 15 --------",
                  "NDCG@k:\t%f" % ndcg_at_k(test, all_predictions, col_user = usercol, col_item = itemcol, col_rating = 'Rating', col_prediction='prediction', k=15),
                  "Recall@k:\t%f" % precision_at_k(test, all_predictions, col_user = usercol, col_item = itemcol, col_rating = 'Rating', col_prediction='prediction', k=15),
                  "MNAP@K:\t%f" % mean_average_precision_at_k(model,train, test, K=15), 
                  "-------- K = 20 --------",
                  "NDCG@k:\t%f" % ndcg_at_k(test, all_predictions, col_user = usercol, col_item = itemcol, col_rating = 'Rating', col_prediction='prediction', k=20),
                  "Recall@k:\t%f" % precision_at_k(test, all_predictions, col_user = usercol, col_item = itemcol, col_rating = 'Rating', col_prediction='prediction', k=20),
                  "MNAP@K:\t%f" % mean_average_precision_at_k(model,train, test, K=20), sep = '\n')

In [ ]:
all_predictions = compute_ranking_predictions(svd, train, usercol = 'userID', itemcol = 'itemID', remove_seen=True)

In [ ]:
all_predictions.head()